In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Functions we have developed in earlier assignnments

In [ ]:
def get_outliers_df(df: pd.DataFrame, ignore_cols = []):
    result = pd.DataFrame()
    ignore_cols = set(ignore_cols)
    for col in df.columns.values:
        if col in ignore_cols:
            continue
        q1, q3 = np.percentile(df[col].to_numpy(), [25,75])
        iqr = q3-q1
        lower_boundary = q1 - (1.5*iqr)
        upper_boundary = q3 + (1.5 * iqr)
        upper_outliers = df.query(f'{col} > {upper_boundary}')[col]
        lower_outliers = df.query(f'{col} < {lower_boundary}')[col]
        result[f'{col}_outliers'] = pd.concat([upper_outliers, lower_outliers])
    return result
        

In [ ]:
from sklearn.linear_model import LinearRegression

def random_imputation(df_to_impute: pd.DataFrame, features: list[str]) -> pd.DataFrame:
    copy = df_to_impute.copy()
    for feature in features:
        copy[feature + '_imp'] = copy[feature]
        number_missing = copy[feature].isnull().sum()
        observed_values = copy.loc[copy[feature].notnull(), feature]
        copy.loc[copy[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    return copy

def bootstrapped_stochastic_imputation(df_to_impute: pd.DataFrame, features: list[str]): 
    copy = df_to_impute.copy()
    df = random_imputation(copy, features)
    random_data = pd.DataFrame(columns = ["Ran" + name for name in features])

    for feature in features:
            
        random_data["Ran" + feature] = df[feature + '_imp']
        parameters = list(set(df.columns) - set(features) - {feature + '_imp'})
        
        model = LinearRegression()
        model.fit(X = df[parameters], y = df[feature + '_imp'])
        
        predict = model.predict(df[parameters])
        std_error = (predict[df[feature].notnull()] - df.loc[df[feature].notnull(), feature + '_imp']).std()
        
        random_predict = np.random.normal(size = df[feature].shape[0], 
                                        loc = predict, 
                                        scale = std_error)
        random_data.loc[(df[feature].isnull()) & (random_predict > 0), "Ran" + feature] = random_predict[(df[feature].isnull()) & 
                                                                                (random_predict > 0)]
    for feature in features:
        copy[feature].fillna(random_data[f'Ran{feature}'],inplace=True)

    return copy


In [ ]:

from pandas.io.formats.info import DataFrameInfo

def get_missing_count_ratio_df(df: pd.DataFrame):
    row_count, col_count = df.shape
    info = DataFrameInfo(data = df)
    info_df= pd.DataFrame(
        {'Non-Null Count': info.non_null_counts, 'Dtype': info.dtypes}
    )

    # Calculating missing data per column. 
    info_df['Missing Count'] = row_count- info_df['Non-Null Count']
    info_df['Missing Ratio'] = (info_df['Missing Count'] / row_count).astype(float)
    # Sorting missing data from highest % to lowest %
    return info_df.sort_values(by=['Non-Null Count'], ascending=True)

# 1. Analysis: Frame the problem and look at the big picture.

## Objective
To create a regression model that can predict house sales prices based on different features of the house. This can help people to determine if they are selling their house at the correct price. 

## Framing
This will utilize supervised learning, since we are doing regression to predict the sales price and the sales price is known for each house.

## Performance Measuring
Performance will be measured by different metrics related to regression such as coeffecient of determination ($R^2$) and Mean Squared Error (MSE).

# 2. Get the data

The dataset is from a Kaggle competition and can be found here: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv.

You must join the competition in order to get access to the data.

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

### 3. Explore the data

In [ ]:
eda = data.copy()

In [ ]:
eda.columns

## Exploring Missing Values

In [ ]:
get_missing_count_ratio_df(eda).head(n=10)

We see that there are features where almost all the data is missing such as PoolQC, MiscFeature, Alley and more. We will now look further into the features where a signifcant part of the data is missing to attempt to get an understanding of why they are missing

Looking at the rows where it is not null

In [ ]:
eda.loc[~eda["PoolQC"].isna()]["PoolQC"].value_counts()

Only 7 rows have a value for Pool Quality, so the missing values might be because the house does not have a pool. We can verify this by looking at the PoolArea feature for the rows where PoolQC is missing.

In [ ]:
eda.loc[eda["PoolQC"].isna()][["PoolQC", "PoolArea"]]

In [ ]:
eda.loc[eda["PoolQC"].isna()][["PoolQC", "PoolArea"]].query("PoolArea > 0")

PoolQC is missing because they do not have a pool. According to `data_description.txt` this should have been set to `NA`, so we can replace the missing values with `NA` during preprocessing.

Now we will take a look at `MiscFeature`. `MiscFeature` is Miscellaneous feature not covered in other categories such as Elevator, 2nd Garage, Tennis Court and more.

In [ ]:
eda.loc[~eda["MiscFeature"].isna()]["MiscFeature"].value_counts()

This feature has missing values because the house might not have any Miscellaneous features. We will verify this by checkign the `MiscVal` feature that indicates the value of the feature in dollars. 

In [ ]:
eda.loc[eda["MiscFeature"].isna()][["MiscFeature", "MiscVal"]]

In [ ]:
eda.loc[eda["MiscFeature"].isna()][["MiscFeature", "MiscVal"]].query("MiscVal > 0")

It would seem like `MiscFeature` has missing values because the house has no Miscellaneous features and they have not been put down as `NA` as specified in `data_description.txt`

In [ ]:
eda.loc[~eda["Alley"].isna()]["Alley"].value_counts()

We notice that `Alley` also should have been classified as NA but has been put down as null instead.

The `Fence`, `MasVnrType`, `FireplaceQu` feature has missing values for the same reason as `PoolQC`, `MiscFeatures` and `Alley`

## Exploring Outliers

We first select all numerical features

In [ ]:
eda_numerical = eda.select_dtypes([float, int])

In [ ]:
import math
def boxplot_columns(df: pd.DataFrame, plots: int):
    df = df.loc[:, df.max().sort_values(ascending=False).index]
    column_bins = np.array(list(map(lambda x: math.floor(x), np.linspace(0, len(df.columns.values), num=plots))))
    prev = 0
    curr = 1
    for column_idx in column_bins[1:]:
        # curr = column_idx if curr == 0 else column_idx - 1
        curr = column_idx 
        plt.figure(figsize=(15,10))
        df[df.columns.values[prev:curr]].boxplot()
        plt.show()
        prev = curr

In [ ]:
import math
def plot_histograms(df: pd.DataFrame, plots: int = 5):
    df = df.loc[:, df.max().sort_values(ascending=False).index]
    column_bins = np.array(list(map(lambda x: math.floor(x), np.linspace(0, len(df.columns.values), num=plots))))
    prev = 0
    curr = 1
    for column_idx in column_bins[1:]:
        # curr = column_idx if curr == 0 else column_idx - 1
        curr = column_idx
        plt.figure(figsize=(15,10))
        df[df.columns.values[prev:curr]].hist()
        plt.show()
        prev = curr

### Looking at the boxplots for all numerical features

The target is `SalePrice`, so we drop that before plotting.

In [ ]:
boxplot_columns(eda_numerical.drop('SalePrice', axis=1), 5)

Initially when looking at the boxplots, we notice that a lot of the features have outliers, including `LotArea, MiscVAl, TotalBsmtSF, BsmtFinSF1, GrLivArea, lstFlSF, BsmtUnfSF, MasVnrArea, BsmtFinSF2, GarageArea, WoodDeckSF, PoolArea, LowQualFinSF, EnclosedPorc, OpenPorchSF, 3SsnPorch, ScreenPorch, LotFrontage, MSSubClass, TotRmsAbcGrid, OverallCond, BedRoomAbvGr, GarageCars, Fireplaces, KitchenAbvGr, BsmtFullBath, BstmfHalfBath`. At this moment, we are unsure if they are natural outliers or if they represent data entry errors or processing errors. We take a closer look to see if we can learn more about the outliers. Some features are rare for common houses, meaning when a certain price class is reached the feature suddently becomes common. 

A lot of the features with outliers have IQR close to 0 and then outliers above the upper tick. This can be explained by many houses does not have the feature that the feature is describing, thus making the other houses that actually have the features to outliers. E.g. Most houses do not have an `EnclosedPorch`, but the few that do, will then be outliers. This is related to the points we made above. Some features are also related to the price 

Out of the boxplots above, we can see that many features have outliers, but we are not sure if they are natural outliers or if they are data entry errors or processing errors. We will take a closer look at the features with outliers to see if we can learn more about them. 

In [ ]:
eda_numerical['SalePrice'].hist()
plt.show()

When looking at the distribution of the SalePrice it looks positive skewed, which would seem naturally. Only a few percentage of the houses are very expensive, while most houses are in the lower to middle class.
This can also explain some of our outliers. They are natural outliers, because the feature maybe is inherently tied to the price range of the house. We can verify this by taking a look at the correlation matrix and compare the distribution of one of the features that is higly correlated to the Sale Price. 

We will try to see plot the correlations to see 

In [ ]:
import seaborn as sns
corr = eda_numerical.corr()
plt.figure(figsize=(25,15))

sns.heatmap(corr)
plt.show()


Here we see that `GrLivArea` is positively correlated with `SalePrice`, so we would assume that `GrLivArea` follow a similar distribution to the `SalePrice`, thus we can assume that the `GrLivArea` outliers are natural.  
The same reasoning can be applied to the other features with outliers - They are either rare features of a house such as `EnclosedPorch` or they are correlated to the price class of the house such as `GrLivArea` and `OverallQual`

In [ ]:
eda_numerical[['GrLivArea', 'SalePrice', 'OverallQual']].hist()
plt.show()

From the observations we have made, we deem the outliers to be natural and we will not remove them from the dataset.

### Distributions of the features.

In [ ]:
plot_histograms(eda_numerical, plots=19)

We notice that some of the features are positive-skwewed. Of the postiive-skewed features, some of them are very positive-skewed such as `WoodDeckSF` and `OpenPorchSF`, while some of them are only slightly positive-skewed. For the very positive-skewed features we can apply a log(n) transformation and for the data that is only slightly positive-skewed we can apply a sqrt(n) transformation. 

When training the models, we will first train the model without the transformations and then after apply the transformations and retrain, to see if it makes an siginificant impact. 

## Target

The target / y in this dataset is the `SalePrice` features. 

# 4. Prepare the data

In [ ]:
data_cleansed = data.copy()

## Data Cleaning

We will clean missing data we identified in the exploratory data analysis. The missing data was caused by `NA` being loaded as nulls, so we will replace the missing data with `NA` or something similar. 

In [ ]:
get_missing_count_ratio_df(eda).head(n=20)

In [ ]:
# Fills it nulls with NA string for the features where they can have a NA category according to data_description.txt
def fill_missing_categorical_data(df: pd.DataFrame, features: list[str] = ['PoolQC', 'MiscFeature', 'Alley','Fence', 'MasVnrType', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCond','GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual']) -> pd.DataFrame:
    for feature in features:
        df[feature] = df[feature].fillna(f'NA_{feature}')
    return df

# Fills out nulls with 0 for features where they are null because they are missing, e.g. for MasVnrArea, where the house might not have Masonry Veneer, so the area should be 0.
def fill_missing_numerical_data(df: pd.DataFrame, features: list[str] = ['MasVnrArea', 'LotFrontage']) -> pd.DataFrame:
    for feature in features:
        df[feature] = df[feature].fillna(0)
    return df



In [ ]:
data_cleansed = fill_missing_numerical_data(fill_missing_categorical_data(data_cleansed))
data_cleansed = data_cleansed.dropna(subset=['Electrical'])

In [ ]:
get_missing_count_ratio_df(data_cleansed).head(n=10)

We know from the correlation matrix that GarageYrBlt is highly correlated with BltYr, so we handle the missing values by dropping the columns entirely in the feature selection section 

## Feature Selection

Dropping `GarageYrBlt` and `Id` features

In [ ]:
data_cleansed = data_cleansed.drop(columns=['GarageYrBlt', 'Id'], axis=1)
data_cleansed

## Feature Engineering

We have a mix of categorical and numerical features. 

First we onehot encode the categorical features.

In [ ]:
def onehot_encode_categorical_features(df: pd.DataFrame) -> pd.DataFrame:
    return pd.get_dummies(df, dtype='int')

In [ ]:
data_cleansed = onehot_encode_categorical_features(data_cleansed)
data_cleansed

Currenty our dataset has 302 features and we would like to reduce the amount of features before training the models. We can reduce the features by manually dropping highly correlated features using the correlation matrix, by Lasso Regression or by using PCA.  

In [ ]:
def get_Xy(df: pd.DataFrame):
    return df.drop('SalePrice', axis=1), df['SalePrice']

We assume Lasso Regression is better suited for the task, since PCA only reduces feature space by retaining the maximum variance, while Lasso Regression performs feature selection based by feature importance. Features that do not have high importance in regards to the target wills shrink to zero, leaving the important features back. 

Before we perform Lasso Regression, we must first scale the data to ensure all features are regularized correctly.

## Feature Scaling

To scale the data we use a Standard Scaler, since it is less sensitive to outliers.

In [ ]:
X,y = get_Xy(data_cleansed)

In [ ]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()

X_scaled = standardScaler.fit_transform(X)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

hyperparam_grid = {'alpha': [0.01, 0.1, 1, 5, 10, 20], 'fit_intercept': [True, False]}
lasso_gs = GridSearchCV(estimator=Lasso(), param_grid=hyperparam_grid)
lasso_gs.fit(X_scaled,y)

In [ ]:
feature_names = X.columns
feature_names

In [ ]:
lasso_estimator = lasso_gs.best_estimator_
lasso_estimator.coef_


In [ ]:
lasso_estimator.coef_.reshape(-1, 1)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

selected_features_df = pd.DataFrame()
for idx, feature in enumerate(X.columns.values):
    selected_features_df[feature] = [lasso_estimator.coef_[idx]]
# selected_features_df[X.columns.values] = lasso_estimator.coef_.reshape(-1, 1)
selected_features_df




In [ ]:
selected_features_df[selected_features_df[X.columns.values] != 0]

After having performed Lasso Regression, we have reduced the features from 301 to 248

In [ ]:
warnings.filterwarnings("default")


# 5. Short-list promising models.

We first quickly try different regression models such as OLS, Random Forest, Gradient Boosted Trees, ElasticNet and K-Nearest Neighbour Regressor

In [ ]:
X = X[selected_features_df.columns.values]
X_scaled = standardScaler.fit_transform(X)
y = y 

In [ ]:
X_scaled.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def print_performance_metrics(model, X_train, y_train, X_test, y_test):
    print("R^2 - train:", model.score(X_train, y_train))
    print("R^2 - test:", model.score(X_test, y_test))
    mse_train = mean_squared_error(y_true=y_train, y_pred=model.predict(X_train))
    mse_test = mean_squared_error(y_true=y_test, y_pred=model.predict(X_test))
    print("MSE - train:", mse_train)
    print("MSE - test:", mse_test)
    print("MSE DIF test-train", mse_test-mse_train)

%matplotlib inline
def plot_predictions_to_actual(model, X_train, y_train, X_test, y_test):
    plt.figure(figsize=(15,10))
    plt.scatter(x=[x for x in range(len(X_train))], y=model.predict(X_train))
    plt.title("Training set predictions")
    plt.show()

    plt.figure(figsize=(15,10))
    plt.scatter(x=[x for x in range(len(X_test))], y=model.predict(X_test))
    plt.title("Test set predictions")
    plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=4242)

## Quick prototyping

### OLS

In [ ]:
from sklearn.linear_model import LinearRegression

linearRegression = LinearRegression()

linearRegression.fit(X_train,y_train)

In [ ]:
print_performance_metrics(linearRegression, X_train, y_train, X_test, y_test)

In [ ]:
plot_predictions_to_actual(linearRegression, X_train, y_train, X_test, y_test)

OLS generalize very poorly, but performed well in the training set.

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X_train, y_train)

In [ ]:
print_performance_metrics(rf_regressor, X_train, y_train, X_test, y_test)

Does preform good, but seems a bit overfittet

### Gradient Boosted Tree

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

In [ ]:
print_performance_metrics(xgb, X_train, y_train, X_test, y_test)

Performs a bit better than the Random Forest Regressor. Still overfitted, but the MSE on the test set is lower. It seems to be generalizing better.

### Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)

In [ ]:
print_performance_metrics(elastic_net, X_train, y_train, X_test, y_test)

Performs nearly as good as the Gradient Boosted Tree and the Random Forest, but does not overfit. Generalizes well to our dataset.

### K-Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

In [ ]:
warnings.filterwarnings('ignore')
print_performance_metrics(knn, X_train, y_train, X_test, y_test)